# Importing your own neural net

Verifying the example neural net was all well and good, but you probably want to verify your own neural net now. In this tutorial, we show you how to import the parameters for a feed-forward neural net with an architecture of your choice.

In [1]:
using MIPVerify
using Gurobi
using MAT

We'll download a `.mat` file containing the parameters of a sample neural net containing three layers (exported from `tensorflow`). 

In [2]:
param_dict = Base.download("https://github.com/vtjeng/MIPVerify_data/raw/master/weights/mnist/n2.mat") |> matread

Dict{String,Any} with 26 entries:
  "fc1/weight"           => Float32[0.000222324 6.78186f-6 … 0.00111799 -0.0005…
  "fc3/weight/Adam_1"    => Float32[4.4054f-5 7.04086f-5 … 2.05744f-5 1.5445f-5…
  "logits/bias/Adam_1"   => Float32[9.47849f-6 1.14659f-5 … 2.12202f-5 2.5675f-…
  "fc1/bias"             => Float32[0.675125 -0.372304 … 0.540256 -0.468334]
  "fc3/bias"             => Float32[0.239015 1.05777 … 1.87256 1.10636]
  "logits/weight/Adam_1" => Float32[9.73416f-5 7.37292f-5 … 0.000153108 0.00013…
  "fc2/weight/Adam_1"    => Float32[0.000456424 0.000191762 … 9.0507f-5 5.83681…
  "fc2/bias"             => Float32[1.89861 1.58582 … -0.54874 1.00736]
  "fc3/bias/Adam_1"      => Float32[3.96812f-6 6.44411f-6 … 4.03203f-6 1.96784f…
  "logits/bias/Adam"     => Float32[-0.00108494 0.000629807 … 0.000172997 0.001…
  "beta1_power"          => 0.0
  "fc2/bias/Adam"        => Float32[0.00093958 0.000148308 … -0.000481088 -0.00…
  "logits/bias"          => Float32[-0.167159 0.670988 … -0.16360

## Layer 1

Let's begin by importing the parameters for the first fully connected layer, which has 784 inputs (corresponding to a flattened 28x28 image) and 24 outputs.

### Basic Approach

We begin with a basic approach where we extract the weights and the biases of the fully connected layer seperately.

In [3]:
fc1_weight = param_dict["fc1/weight"]

784×24 Array{Float32,2}:
  0.000222324   6.78186f-6    0.000211635  …   0.00111799   -0.00055723 
  0.000270549  -0.000155775   0.000345145      0.0011448    -0.000638638
  0.00157308    0.00294467    0.00133911      -0.00275523    0.00436651 
  0.00124757   -4.55079f-5    0.000364477      0.000844778   0.00030634 
  0.000181523   7.54999f-5    0.000184328      0.000718589  -0.000354625
  0.00100992   -0.00056507    0.000376461  …   0.00108511   -2.14134f-5 
  0.0011267     0.00277671    0.00199021      -0.00308363    0.0037927  
  0.00172865    0.00107658    0.000331343     -0.000169128   0.00168143 
 -0.00152659   -0.0020568     0.000401156      0.00151572   -0.000661888
  0.000884574  -0.000527718   0.000382487      0.00106229   -4.86445f-5 
  0.00147657    0.00275898    0.00151834   …  -0.00248229    0.00379858 
 -3.01078f-5    0.000446275   0.000935425     -0.00300283    0.00124241 
  2.6066f-5     0.000116717   3.48966f-6      -0.000166195   0.000101932
  ⋮                       

In [4]:
fc1_bias = param_dict["fc1/bias"]

1×24 Array{Float32,2}:
 0.675125  -0.372304  -0.202615  …  -0.0190356  0.540256  -0.468334

We group the weights and biases in a `Linear`.

_(NB: We have to flatten the bias layer using `squeeze` since `Linear` expects a 1-D array for the bias.)_

In [5]:
fc1_manual = Linear(fc1_weight, squeeze(fc1_bias, 1))

Linear(784 -> 24)

That was a lot to remember. Wouldn't it be nice if there was a helper function to take care of all that?

### With Helper Functions

In [6]:
fc1 = get_matrix_params(param_dict, "fc1", (784, 24))

Linear(784 -> 24)

`get_matrix_params` requires that 1) you specify the expected size of the layer, and 2) your weight and bias arrays following the naming convention outlined in the [documentation](https://vtjeng.github.io/MIPVerify.jl/stable/utils/import_weights.html#MIPVerify.get_matrix_params-Tuple{Dict{String,V} where V,String,Tuple{Int64,Int64}}).

As a sanity check, you can verify that the parameters we get from both methods are equal.

In [7]:
fc1_manual == fc1

true

## Importing the rest of the layers

Since we followed the naming convention required by `get_matrix_params` when exporting our neural net parameters as a `.mat` file, importing the rest of the neural net is relatively straightforward.

In [8]:
fc2 = get_matrix_params(param_dict, "fc2", (24, 24))

Linear(24 -> 24)

In [9]:
fc3 = get_matrix_params(param_dict, "fc3", (24, 24))

Linear(24 -> 24)

In [10]:
logits = get_matrix_params(param_dict, "logits", (24, 10))

Linear(24 -> 10)

## Composing the network

We now put the entire network together. We need to flatten the input since the input images are provided as a 4-dimensional tensor. (Note that there is no `ReLU` after the softmax layers).

In [11]:
nn = Sequential([
        Flatten(4),
        fc1,
        ReLU(),
        fc2,
        ReLU(),
        fc3,
        ReLU(),
        logits
    ], "MNIST.n2")

sequential net MNIST.n2
  (1) Flatten(): flattens 4 dimensional input, with dimensions permuted according to the order [4, 3, 2, 1]
  (2) Linear(784 -> 24)
  (3) ReLU()
  (4) Linear(24 -> 24)
  (5) ReLU()
  (6) Linear(24 -> 24)
  (7) ReLU()
  (8) Linear(24 -> 10)


## Verifying that you imported the network correctly
It's important to make sure that you imported the network correctly. We do this by passing in images from the test set.

In [12]:
mnist = read_datasets("MNIST")
MIPVerify.frac_correct(nn, mnist.test, 10000)

Computing fraction correct...100%|██████████████████████| Time: 0:00:04


0.9706

## Finding and Adversarial Example

Finally, we find an adversarial example for a sample input.

In [13]:
sample_image = MIPVerify.get_image(mnist.test.images, 1);

In [14]:
MIPVerify.find_adversarial_example(nn, sample_image, 4, GurobiSolver())

[notice | MIPVerify]: Attempting to find adversarial example. Neural net predicted label is 8, target labels are [4]
[notice | MIPVerify]: Loading model from cache.


Dict{Any,Any} with 10 entries:
  :PerturbationFamily => unrestricted
  :TargetIndexes      => [4]
  :SolveStatus        => :Optimal
  :TotalTime          => 453.259
  :TighteningApproach => "loaded_from_cache"
  :Output             => JuMP.GenericAffExpr{Float64,JuMP.Variable}[0.106403902…
  :PredictedIndex     => 8
  :Model              => Minimization problem with:…
  :Perturbation       => JuMP.Variable[__anon__ __anon__ __anon__ __anon__ __an…
  :PerturbedInput     => JuMP.Variable[__anon__ __anon__ __anon__ __anon__ __an…

Academic license - for non-commercial use only
Optimize a model with 3433 rows, 3280 columns and 46856 nonzeros
Variable types: 3208 continuous, 72 integer (72 binary)
Coefficient statistics:
  Matrix range     [2e-07, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+02]
  RHS range        [4e-03, 6e+02]

MIP start did not produce a new incumbent solution
MIP start violates constraint R1072 by 2.000000000

Presolve removed 2978 rows and 2237 columns
Presolve time: 0.12s
Presolved: 455 rows, 1043 columns, 40972 nonzeros
Variable types: 971 continuous, 72 integer (72 binary)

Root relaxation: objective 0.000000e+00, 266 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    8          -    0.00000      -     -    0s
Another try with MIP start
     0     0    0.00000    0    7          -    0.00000      -     -    0

There we go! Now it's your turn to try to verify your own neural network.